In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import libraries and Data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from itertools import product
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestClassifier
from xgboost import plot_importance

In [ ]:
train = pd.read_csv("../input/competitive-data-science-predict-future-sales/sales_train.csv")
test = pd.read_csv("../input/competitive-data-science-predict-future-sales/test.csv")
item_category = pd.read_csv("../input/competitive-data-science-predict-future-sales/item_categories.csv")
item = pd.read_csv("../input/competitive-data-science-predict-future-sales/items.csv")
shop = pd.read_csv("../input/competitive-data-science-predict-future-sales/shops.csv")

# A Baseline Model with LSTM

In [ ]:
'''dataset = train.pivot_table(index = ['shop_id','item_id'],values = ['item_cnt_day'],columns = ['date_block_num'],fill_value = 0, aggfunc='sum')
dataset.reset_index(inplace = True)
dataset.head()'''

In [ ]:
'''test.head()'''

In [ ]:
'''dataset = pd.merge(test,dataset,on = ['item_id','shop_id'],how = 'left')
dataset.fillna(0,inplace = True)
dataset.head()'''

In [ ]:
'''dataset.drop(['shop_id','item_id','ID'],inplace = True, axis = 1)
dataset.head()'''

In [ ]:
'''dataset.shape'''

In [ ]:
'''# X we will keep all columns execpt the last one 
X_train = np.expand_dims(dataset.values[:,:-1],axis = 2)
# the last column is our label
y_train = dataset.values[:,-1:]

# for test we keep all the columns execpt the first one
X_test = np.expand_dims(dataset.values[:,1:],axis = 2)

# lets have a look on the shape 
print(X_train.shape,y_train.shape,X_test.shape)'''

In [ ]:
'''from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout'''

In [ ]:
'''baseline_model = Sequential()
baseline_model.add(LSTM(units = 64,input_shape = (33,1)))
baseline_model.add(Dense(1))

baseline_model.compile(loss = 'mse',optimizer = 'adam', metrics = ['mean_squared_error'])
baseline_model.summary()'''

In [ ]:
'''baseline_model.fit(X_train,y_train,batch_size = 4096,epochs = 10)'''

In [ ]:
'''# creating submission file 
submission_pfs = baseline_model.predict(X_test)
# creating dataframe with required columns 
submission = pd.DataFrame({'ID':test['ID'],'item_cnt_month':submission_pfs.ravel()})
# creating csv file from dataframe
submission.to_csv('sub_pfs.csv',index = False)'''

In [ ]:
'''submission.head()'''

### Score of the Baseline Model 1.02207

# Data Exploration and feature extraction

In [ ]:
def basic_EDA(df):
    print("-------Sample data---------")
    print(df.head())
    print("-------Description---------")
    print(df.describe())
    print("-------Information---------")
    print(df.info())
    print("-------Columns---------")
    print(df.columns)
    print("-------Data Types---------")
    print(df.dtypes)
    print("-------NULL and NA Values---------")
    print(df.isnull().sum())
    print(df.isna().sum())
    print("-------Shape---------")
    print(df.shape)

## Train

In [ ]:
basic_EDA(train)

In [ ]:
train['date'] = pd.to_datetime(train['date'],format = '%d.%m.%Y')

In [ ]:
train.head()

In [ ]:
train.dtypes

In [ ]:
boxplot = train.boxplot(column=['item_cnt_day'])

In [ ]:
boxplot = train.boxplot(column=['item_price'])

In [ ]:
train[train.item_price > 100000]

In [ ]:
train[train.item_price < 0]

In [ ]:
train[train.item_cnt_day >= 1000]

In [ ]:
train[train.item_cnt_day < 0]

In [ ]:
#Let's check if the neg item_cnt_day are return?
'''neg = train[train.item_cnt_day < 0]
neg.head()

pos = train[train.item_cnt_day > 0]
pos.head()'''
#A function to check if the -ve item_cnt_day are return or not, but this function takes along time to run
#need to optimise it in some way so that it can be run in a reasonable amount of time
#Untill that time we will remove the -ve and continue as such.
'''return_counter = 0
for ind in neg.index:
    for i in range(ind):
        if ((neg['shop_id'][ind] == pos['shop_id'][i]) and 
        (neg['item_id'][ind] == pos['item_id'][i]) and 
        (neg['item_cnt_day'][ind] + pos['item_cnt_day'][i] == 0)):
            return_counter = return_counter + 1
        
            
print(return_counter)'''

In [ ]:
train = train[(train.item_price < 300000 ) & (train.item_price > 0) & (train.item_cnt_day < 1000) & (train.item_cnt_day > 0)]
train.reset_index(inplace = True)
train.head()

## Shop

In [ ]:
basic_EDA(shop)

Transation using Google Translation

!Якутск Орджоникидзе, 56 фран	->	! Yakutsk Ordzhonikidze, 56 francs (Yakutsk is a city)

Адыгея ТЦ "Мега"	->	Adygea shopping center "Mega" (Adygea is a city)

Волжский ТЦ "Волга Молл"	->	Volzhsky shopping center "Volga Mall" (Volzhsky is a city)

**Thus there is a pattern of "city name" + "Type of shop"** <br/>
**We can perform feature extraction from this**

In [ ]:
shop["city"] = shop.shop_name.str.split(" ").map( lambda x: x[0] )
shop["type"] = shop.shop_name.str.split(" ").map( lambda x: x[1] )

In [ ]:
shop.head()

In [ ]:
shop['city'].unique()

In [ ]:
shop['type'].unique()

In [ ]:
shop["shop_type"] = LabelEncoder().fit_transform( shop.type )
shop["shop_city"] = LabelEncoder().fit_transform( shop.city )
shop.head()

In [ ]:
shop = shop[['shop_id', 'shop_city', 'shop_type']]
shop.head()

## Item Category

In [ ]:
basic_EDA(item_category)

***Translation***
1. PC - Headsets / Headphones
1. Accessories - PS2 1
1. Accessories - PS3 2
1. Accessories - PS4 3
1. Accessories - PSP <br/>

**It looks like the 'item_category_name' is 'Type of item - Item name'**

In [ ]:
print(item_category['item_category_name'].str.split('-').map(lambda x: x[0]))
#print(item_category['item_category_name'].str.split('-').map(lambda x: x[1]))

In [ ]:
item_category

**So there are some which does not have the '-'**

**So let's Google translate them and see what they are**
1. Service
1. Blank media (spire)
1. Blank media (piece)
1. Batteries

**So they have the item type and not the specific item**

In [ ]:
print(item_category['item_category_name'].str.split('-').map(lambda x: x[1].strip() if len(x) > 1 else x[0].strip()))

In [ ]:
item_category['item_type'] = item_category['item_category_name'].str.split('-').map(lambda x: x[0])
item_category['item_name'] = item_category['item_category_name'].str.split('-').map(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())
item_category.head()

In [ ]:
item_category['item_type'].unique()

**Translation:** array (['PC', 'Accessories', 'Tickets (Digit)', 'Delivery of goods',
       'Game Consoles', 'Games', 'Android Games', 'MAC Games',
       'PC Games', 'Payment Cards (Cinema, Music, Games)', 'Payment Cards',
       'Cinema', 'Books', 'Music', 'Gifts', 'Programs',
       'Service', 'Service', 'Net media (spire)',
       'Blank media (piece)', 'Batteries'], dtype = object)

In [ ]:
item_category['item_name'].unique()

**Translation:** array (['Headsets / Headphones', 'PS2', 'PS3', 'PS4', 'PSP', 'PSVita',
       'XBOX 360', 'XBOX ONE', 'Tickets (Digit)', 'Delivery of goods',
       'Other', 'Accessories for games', 'Number', 'Additional editions',
       'Collector's Editions', 'Standard Editions',
       'Payment cards (Cinema, Music, Games)', 'Live!', 'Live! (Numeral)',
       'PSN', 'Windows (Digital)', 'Blu', 'DVD', 'Collectible',
       'Artbooks, encyclopedias', 'Audiobooks', 'Audiobooks (Digital)',
       'Audiobooks 1C', 'Business literature', 'Comics, manga',
       'Computer literature', 'Methodical materials 1C', 'Postcards',
       'Educational literature', 'Guides', 'Fiction', 'Local CD',
       'Brand CD', 'MP3', 'Vinyl', 'Music Video',
       'Gift Editions', 'Attributes', 'Gadgets, Robots, Sports',
       'Stuffed Toys', 'Board Games',
       'Board games (compact)', 'Postcards, stickers', 'Development',
       'Certificates, services', 'Souvenirs', 'Souvenirs (hinged)',
       'Bags, Albums, Mouse pads', 'Figures', '1C: Enterprise 8',
       'MAC (Digit)', 'Home & Office', 'Home & Office (Digit)',
       'Educational', 'Educational (Digit)', 'Service', 'Tickets',
       'Blank media (spire)', 'Blank media (piece)',
       'Batteries'], dtype = object)

In [ ]:
item_category['item_type_code'] = LabelEncoder().fit_transform(item_category.item_type)
item_category['item_name_code'] = LabelEncoder().fit_transform(item_category.item_name)
item_category.head()

In [ ]:
item_category = item_category[['item_category_id', 'item_type_code', 'item_name_code']]
item_category.head()

## Item

In [ ]:
basic_EDA(item)

In [ ]:
list(item.item_name.unique())

 '! IN THE POWER OF NAVIGATION (PLAST.) D ',<br /> 
 '! ABBYY FineReader 12 Professional Edition Full [PC, Digital]',<br /> 
 '*** IN THE RAYS OF GLORY (UNV) D',<br /> 
 '*** BLUE WAVE (Univ) D',<br /> 
 '*** BOX (GLASS) D',<br /> 
 '*** NEW AMERICAN GRAFFITI (UNI) D',<br /> 
........................................<br/>
 '// MONGOL S. Bodrov (Region)',<br /> 
 '// NOT LEAVING A TRACE (Region)',<br /> 
 '/ BOMB FOR THE BRIDE / 2DVD / D',<br /> 
 '/ GOLDEN COLLECTION m / f-72',<br /> 
 '/ ONCE IN CHINA-2',<br /> 
 '/THE LAST CHANCE',<br /> 
 '/ THE CURSE OF EL CHARRO',<br /> 
 '/ NORTH AND SOUTH /CH.2/',<br /> 
 '/ DEADLINE LAYOUT',<br /> 
 '/YOU ARE DEAD',<br /> 
 '/ MULTIPLIER SADNESS vol. 2 (ser. 3-4)',<br /> 
 '007 Legends [PS3 Russian Version]',<br /> 
 '007 Legends [PC, Jewel, Russian version]',<br /> 
 '10 YEARS LATER (BD) ',<br /> 
 '10 YEARS LATER (region) ',<br /> 
 'THE 10 MOST POPULAR MOVIES OF THE XX CENTURY 10DVD (rem)',<br /> 
 '10 MOST POPULAR DOMESTIC COMEDIES OF THE XX CENTURY (BD) ',<br /> 
 '100 Best classical melodies (mp3-CD) (Digipack)',<br /> 
 '100 Best relax melodies (mp3-CD) (CD-Digipack)',<br /> 
 '100 Best romantic melodies (mp3-CD) (Digipack)',<br /> 
 ... <br/>
 '1C: Audiobooks. Vagin I.O. Human Resource Management [PC, Digital version] ', <br /> 
 '1C: Audiobooks. Vagin I.O., Glushchay A. Teach yourself to laugh [PC, Digital version] ',<br /> 
 '1C: Audiobooks. Vagin I.O. Joy Management [PC, Digital] ',<br /> 
 '1C: Audiobooks. Vasil Bykov. Go and not come back [PC, Digital] ',<br /> 
 '1C: Audiobooks. Great businessmen or how money was made in the twentieth century (Digipack) ',<br /> 
 '1C: Audiobooks. Verne J. 20,000 Leagues Under the Sea ',<br /> 
 '1C: Audiobooks. Verne J. Children of Captain Grant ',<br /> 
 '1C: Audiobooks. Fun lessons for the little ones. ABC and Account (Jewel) ',<br /> 
 ....<br/>
 '1C: Collection of toys "Behind Enemy Lines" (Steam version) [PC, Digital version]',<br/>
 '1C: Collection of toys "Behind Enemy Lines: Saboteurs 2" [PC, Digital version]',<br/>
 '1C: Collection of toys "Behind Enemy Lines: Saboteurs" [PC, Digital version]',<br/>
 '1C: Collection of toys "Vivisector. The beast inside" [PC, Digital version]',<br/>
 '1C: Collection of toys "Luntik. Preparing for school" [PC, Digital version]',<br/>
 '1C: Collection of toys "Blood Magic: Shadow Time" [PC, Digital]',<br/>
 '1C: Collection of toys "Madagascar"',<br/>
 '1C: Collection of toys "Well, wait! Issue 3. Song for a hare" [PC, Digital version]',<br/>
 '1C: Rex and the Wizards Toy Collection [PC, Digital]',<br/>
 '1C: Rex and the Time Machine Toy Collection [PC, Digital]',,<br />
 <br/>
**SO NO consistency in the text data like that of item_category, there are small consistency, maybe we can use those with the help of NLP**

In [ ]:
item.shape

In [ ]:
def clean_text(item_name):
    item_name = item_name.lower()
    item_name = re.sub(r'[^\w\s]', '', item_name)
    item_name = re.sub(r'\d+', '', item_name)
    item_name = re.sub(' +', ' ', item_name)
    item_name = item_name.strip()
    return item_name

In [ ]:
item['clean_item_name'] = item['item_name'].apply(clean_text)
item.head()

In [ ]:
item['sub_name1'] = item['clean_item_name'].str.split(' ').map(lambda x: x[0])
item['sub_name2'] = item['clean_item_name'].str.split(' ').map(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())
'''
item['sub_name3'] = item['clean_item_name'].str.split(' ').map(lambda x: x[2].strip() if len(x) > 2 else x[1].strip() if len(x) > 1 else x[0].strip())
item['sub_name4'] = item['clean_item_name'].str.split(' ').map(lambda x: x[3].strip() if len(x) > 3 else x[2].strip() if len(x) > 2 else x[1].strip() if len(x) > 1 else x[0].strip())
item['sub_name5'] = item['clean_item_name'].str.split(' ').map(lambda x: x[4].strip() if len(x) > 4 else x[3].strip() if len(x) > 3 else x[2].strip() if len(x) > 2 else x[1].strip() if len(x) > 1 else x[0].strip())
'''
item.head()

In [ ]:
item.sub_name1.unique().shape, item.sub_name2.unique().shape

In [ ]:
item.item_id.unique().shape, item.item_category_id.unique().shape

In [ ]:
item['sub_name_1'] = LabelEncoder().fit_transform(item.sub_name1)
item['sub_name_2'] = LabelEncoder().fit_transform(item.sub_name2)
'''item['sub_name_3'] = LabelEncoder().fit_transform(item.sub_name3)
item['sub_name_4'] = LabelEncoder().fit_transform(item.sub_name4)
item['sub_name_5'] = LabelEncoder().fit_transform(item.sub_name5)'''
item.head()

In [ ]:
item = item[['item_id', 'item_category_id', 'sub_name_1', 'sub_name_2']]
item.head()

## Test

In [ ]:
basic_EDA(test)

In [ ]:
len(set(test.item_id) - set(test.item_id).intersection(set(train.item_id)))

In [ ]:
len(set(test.shop_id) - set(test.shop_id).intersection(set(train.shop_id)))

**So there are no new shop_id in test but there are 363 new item_id in test that are not in train.**

# Model Making

In [ ]:
matrix = []
cols = ['date_block_num','shop_id','item_id']
for i in range(34):
    sales = train[train.date_block_num==i]
    matrix.append(np.array(list(product([i], sales.shop_id.unique(), sales.item_id.unique())), dtype='int16'))
    
matrix = pd.DataFrame(np.vstack(matrix), columns=cols)
matrix['date_block_num'] = matrix['date_block_num'].astype(np.int8)
matrix['shop_id'] = matrix['shop_id'].astype(np.int8)
matrix['item_id'] = matrix['item_id'].astype(np.int16)
matrix.sort_values(cols,inplace=True)

In [ ]:
matrix.head()

In [ ]:
matrix.dtypes, matrix.shape

In [ ]:
train['revenue'] = train['item_price'] *  train['item_cnt_day']

In [ ]:
train.head()

In [ ]:
cols = ['date_block_num','shop_id','item_id']
group = train.groupby(['date_block_num','shop_id','item_id']).agg({'item_cnt_day': ['sum']})
group.columns = ['item_cnt_month']
group.reset_index(inplace=True)

In [ ]:
group.head()

In [ ]:
group.dtypes, group.shape

In [ ]:
boxplot = group.boxplot(column = ['item_cnt_month'])

In [ ]:
matrix = pd.merge(matrix, group, on=cols, how='left')
matrix['item_cnt_month'] = (matrix['item_cnt_month']
                            .astype(np.float32)
                            .fillna(0))

In [ ]:
matrix.head(10)

In [ ]:
matrix.dtypes, matrix.shape

In [ ]:
test['date_block_num'] = 34
test['date_block_num'] = test['date_block_num'].astype(np.int8)
test['shop_id'] = test['shop_id'].astype(np.int8)
test['item_id'] = test['item_id'].astype(np.int16)
test.head()

In [ ]:
test.dtypes, test.shape

In [ ]:
matrix = pd.concat([matrix, test], ignore_index = True, keys = cols, sort = False)
matrix.fillna(0, inplace = True)
matrix.head()

In [ ]:
matrix = pd.merge(matrix, shop, on = 'shop_id', how = 'left')
matrix = pd.merge(matrix, item, on = 'item_id', how = 'left')
matrix = pd.merge(matrix, item_category, on = 'item_category_id', how = 'left')
matrix.head()

In [ ]:
matrix.dtypes

In [ ]:
matrix.drop(['ID'], axis=1, inplace=True)
matrix.head()

In [ ]:
matrix['shop_city'] = matrix['shop_city'].astype(np.int8)
matrix['shop_type'] = matrix['shop_type'].astype(np.int8)
matrix['item_category_id'] = matrix['item_category_id'].astype(np.int8)
matrix['sub_name_1'] = matrix['sub_name_1'].astype(np.int16)
matrix['sub_name_2'] = matrix['sub_name_2'].astype(np.int16)
'''matrix['sub_name_3'] = matrix['sub_name_3'].astype(np.int16)
matrix['sub_name_4'] = matrix['sub_name_4'].astype(np.int16)
matrix['sub_name_5'] = matrix['sub_name_5'].astype(np.int16)'''
matrix['item_type_code'] = matrix['item_type_code'].astype(np.int8)
matrix['item_name_code'] = matrix['item_name_code'].astype(np.int8)
matrix.dtypes

In [ ]:
matrix.head()

In [ ]:
matrix['month'] = matrix['date_block_num'] % 12
matrix.head()

In [ ]:
days = pd.Series([31,28,31,30,31,30,31,31,30,31,30,31])
matrix['days'] = matrix['month'].map(days).astype(np.int8)
matrix.head()

In [ ]:
matrix.dtypes

In [ ]:
#item_cnt_month of previous month, previous 2, 3, 6, and 12 month back
def lag_feature(df, lags, col):
    tmp = df[['date_block_num','shop_id','item_id',col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['date_block_num','shop_id','item_id', col+'_lag_'+str(i)]
        shifted[col+'_lag_'+str(i)] = shifted[col+'_lag_'+str(i)].astype(np.float16)
        shifted['date_block_num'] += i
        df = pd.merge(df, shifted, on=['date_block_num','shop_id','item_id'], how='left')
    return df
matrix = lag_feature(matrix, [1,2,3,6,12], 'item_cnt_month')
matrix.head()

In [ ]:
#Average and lag by date_block_num
group = matrix.groupby(['date_block_num']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num'], how='left')
matrix['date_avg_item_cnt'] = matrix['date_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_avg_item_cnt')
matrix.drop(['date_avg_item_cnt'], axis=1, inplace=True)
matrix.head(10)

In [ ]:
'''#Average and Lag by month
group = matrix.groupby(['month']).agg({'item_cnt_month':['mean']})
group.columns = ['month_avg_item_cnt']
group.reset_index(inplace = True)

matrix = pd.merge(matrix, group, on=['month'], how = 'left')
matrix['month_avg_item_cnt'] = matrix['month_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'month_avg_item_cnt')
matrix.head(10)'''

In [ ]:
#Average and Lag by data_block_num and shop_id
group = matrix.groupby(['date_block_num','shop_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_shop_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'shop_id'], how='left')
matrix['date_shop_avg_item_cnt'] = matrix['date_shop_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_shop_avg_item_cnt')
matrix.head(10)

In [ ]:
'''#Average and Lag by month and shop_id
group = matrix.groupby(['month','shop_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'month_shop_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['month', 'shop_id'], how='left')
matrix['month_shop_avg_item_cnt'] = matrix['month_shop_avg_item_cnt'].astype(np.float16)
#matrix = lag_feature(matrix, [1], 'month_shop_avg_item_cnt')
matrix.head(10)'''

In [ ]:
#Average and Lag by data_block_num and item_id
group = matrix.groupby(['date_block_num','item_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_item_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'item_id'], how='left')
matrix['date_item_avg_item_cnt'] = matrix['date_item_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_item_avg_item_cnt')
matrix.head(10)

In [ ]:
'''#Average and Lag by month and item_id
group = matrix.groupby(['month','item_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'month_item_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['month', 'item_id'], how='left')
matrix['month_item_avg_item_cnt'] = matrix['month_item_avg_item_cnt'].astype(np.float16)
#matrix = lag_feature(matrix, [1], 'month_item_avg_item_cnt')
matrix.head(10)'''

In [ ]:
'''#Average and Lag by month and shop_city
group = matrix.groupby(['month','shop_city']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'month_shop_city_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['month', 'shop_city'], how='left')
matrix['month_shop_city_avg_item_cnt'] = matrix['month_shop_city_avg_item_cnt'].astype(np.float16)
#matrix = lag_feature(matrix, [1], 'month_shop_city_avg_item_cnt')
matrix.head(10)'''

In [ ]:
'''#Average and Lag by month and shop_type
group = matrix.groupby(['month','shop_type']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'month_shop_type_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['month', 'shop_type'], how='left')
matrix['month_shop_type_avg_item_cnt'] = matrix['month_shop_type_avg_item_cnt'].astype(np.float16)
#matrix = lag_feature(matrix, [1], 'month_shop_type_avg_item_cnt')
matrix.head(10)'''

In [ ]:
'''#Average and Lag by date_block_num and shop_city
group = matrix.groupby(['date_block_num','shop_city']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_shop_city_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'shop_city'], how='left')
matrix['date_shop_city_avg_item_cnt'] = matrix['date_shop_city_avg_item_cnt'].astype(np.float16)
#matrix = lag_feature(matrix, [1], 'date_shop_city_avg_item_cnt')
matrix.head(10)'''

In [ ]:
'''#Average and Lag by date_block_num and shop_type
group = matrix.groupby(['date_block_num','shop_type']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_shop_type_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'shop_city'], how='left')
matrix['date_shop_type_avg_item_cnt'] = matrix['date_shop_type_avg_item_cnt'].astype(np.float16)
#matrix = lag_feature(matrix, [1], 'date_shop_type_avg_item_cnt')
matrix.head(10)'''

In [ ]:
'''#Average and Lag by month and item_category_id
group = matrix.groupby(['month','item_category_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'month_cat_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['month', 'item_category_id'], how='left')
matrix['month_cat_avg_item_cnt'] = matrix['month_cat_avg_item_cnt'].astype(np.float16)
#matrix = lag_feature(matrix, [1], 'month_cat_avg_item_cnt')
matrix.head(10)'''

In [ ]:
'''#Average and Lag by month and item_type_code
group = matrix.groupby(['month','item_type_code']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'month_item_type_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['month', 'item_type_code'], how='left')
matrix['month_item_type_avg_item_cnt'] = matrix['month_item_type_avg_item_cnt'].astype(np.float16)
#matrix = lag_feature(matrix, [1], 'month_item_type_avg_item_cnt')
matrix.head(10)'''

In [ ]:
'''#Average and Lag by month and item_name_code
group = matrix.groupby(['month','item_name_code']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'month_item_name_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['month', 'item_name_code'], how='left')
matrix['month_item_name_avg_item_cnt'] = matrix['month_item_name_avg_item_cnt'].astype(np.float16)
#matrix = lag_feature(matrix, [1], 'month_item_name_avg_item_cnt')
matrix.head(10)'''

In [ ]:
'''#Average and Lag by data_block_num and item_category_id
group = matrix.groupby(['date_block_num','item_category_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_cat_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'item_category_id'], how='left')
matrix['date_cat_avg_item_cnt'] = matrix['date_cat_avg_item_cnt'].astype(np.float16)
#matrix = lag_feature(matrix, [1], 'date_cat_avg_item_cnt')
matrix.head(10)'''

In [ ]:
'''#Average and Lag by data_block_num and item_type_code
group = matrix.groupby(['date_block_num','item_type_code']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_item_type_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'item_type_code'], how='left')
matrix['date_item_type_avg_item_cnt'] = matrix['date_item_type_avg_item_cnt'].astype(np.float16)
#matrix = lag_feature(matrix, [1], 'date_item_type_avg_item_cnt')
matrix.head(10)'''

In [ ]:
'''#Average and Lag by month and item_name_code
group = matrix.groupby(['date_block_num','item_name_code']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_item_name_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'item_name_code'], how='left')
matrix['date_item_name_avg_item_cnt'] = matrix['date_item_name_avg_item_cnt'].astype(np.float16)
#matrix = lag_feature(matrix, [1], 'date_item_name_avg_item_cnt')
matrix.head(10)'''

In [ ]:
matrix.columns

In [ ]:
matrix.info()

In [ ]:
X_train = matrix[matrix.date_block_num <= 33].drop(['item_cnt_month'], axis=1)
Y_train = matrix[matrix.date_block_num <= 33]['item_cnt_month']
X_valid = matrix[matrix.date_block_num == 33].drop(['item_cnt_month'], axis=1)
Y_valid = matrix[matrix.date_block_num == 33]['item_cnt_month']
X_test = matrix[matrix.date_block_num == 34].drop(['item_cnt_month'], axis=1)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
model = XGBRegressor(
    max_depth=8,
    n_estimators=1000,
    min_child_weight=300, 
    colsample_bytree=0.8, 
    subsample=0.8, 
    eta=0.3,    
    seed=42)

model.fit(
    X_train, 
    Y_train, 
    eval_metric="rmse", 
    eval_set=[(X_train, Y_train), (X_valid, Y_valid)], 
    verbose=True, 
    early_stopping_rounds = 10)

In [ ]:
Y_test = model.predict(X_test).clip(0, 20)

submission = pd.DataFrame({
    "ID": test.index, 
    "item_cnt_month": Y_test
})
submission.to_csv('submission.csv', index=False)

In [ ]:
def plot_features(booster, figsize):    
    fig, ax = plt.subplots(1,1,figsize=figsize)
    return plot_importance(booster=booster, ax=ax)
plot_features(model, (10,14))

### Could use differernt feature generation like based on months of the year and add a categorical feature based on month, and mean based on month and other stuff